In [ ]:
### Converting the narrow metrics wide in the itemized table
# combining lumenad and itemized tables

In [ ]:
WITH cte_imp AS (
SELECT
media_plan_id 
,FlightsDimensionValueID
,SUM(units) AS impressions
,itemized_product_id
FROM i_3
WHERE unit_label ='Impressions' 
GROUP BY FlightsDimensionValueID
),
 cte_click AS (
SELECT
media_plan_id 
,FlightsDimensionValueID
,SUM(units) AS clicks
,itemized_product_id
FROM i_3
WHERE unit_label ='Clicks'
GROUP BY FlightsDimensionValueID
),
cte_views AS (
SELECT
media_plan_id 
,FlightsDimensionValueID
,SUM(units) AS views
,itemized_product_id
FROM i_3
WHERE unit_label ='Views'
GROUP BY FlightsDimensionValueID
),
,cte_lum AS (

flight_dimension_value_id
,base_cost
,spent
,impressions AS a_imp
,clicks AS a_imp
,completed_views_100 AS a_imp
FROM lumenad_data_C
GROUP BY flight_dimension_value_id
)
cte_lum AS (

flight_dimension_value_id
,base_cost
,spent
,impressions AS a_imp
,clicks AS a_imp
,completed_views_100 AS a_imp
FROM lumenad_data_C
GROUP BY flight_dimension_value_id


)

SELECT
i.media_plan_id
,i.itemized_product_id
,i.FlightsDimensionValueID
,i.LineItemsDimensionValueID
,i.ChannelsDimensionValueID
,i.PlatformsDimensionValueID
,i.TacticsDimensionValueID
,i.start_date
,i.end_date
,i.total_days
,impressions
,clicks
,views
,a_imp
,a_clicks
,a_views
,SUM(l.base_cost)
,SUM(l.spent)
,AVG(i.total_margin)
,SUM(i.gross_budget)
,SUM(i.net_budget)
,AVG(i.gross_rate)
,AVG(i.net_rate)
FROM i_3 AS i
LEFT JOIN cte_imp AS imp
ON i.FlightsDimensionValueID = imp.FlightsDimensionValueID
LEFT JOIN cte_click AS clk
ON i.FlightsDimensionValueID = clk.FlightsDimensionValueID
LEFT JOIN cte_views AS v
ON i.FlightsDimensionValueID = v.FlightsDimensionValueID
--WHERE i.media_plan_id ='34'
GROUP BY i.FlightsDimensionValueID
--ORDER BY i.itemized_product_id
ORDER BY i.media_plan_id, i.FlightsDimensionValueID



In [ ]:
### Creating the itemized recusive data
## KPIs aggregated in gross_budget_agg CTE

In [ ]:
WITH RECURSIVE date_range AS (
  SELECT itemized_product_id, 
  DATE(start_date) AS date, 
  DATE(end_date) AS end_date
  FROM `capstone-itemized.1.t1` 
  UNION ALL
  SELECT itemized_product_id, DATE_ADD(date, INTERVAL 1 DAY), end_date
  FROM date_range
  WHERE date < end_date
),  gross_budget_agg AS (
  SELECT 
  temp.itemized_product_id
  ,(AVG(gross_budget)/ COUNT(DISTINCT date)) AS gross_budget
  ,(AVG(net_budget)/ COUNT(DISTINCT date)) AS net_budget
  ,(AVG(p_imp)/ COUNT(DISTINCT date)) AS p_imp
  ,(AVG(p_clicks)/ COUNT(DISTINCT date)) AS p_clicks
  ,(AVG(p_views)/ COUNT(DISTINCT date)) AS p_views
    ,(AVG(a_imp)/ COUNT(DISTINCT date)) AS a_imp
  ,(AVG(a_clicks)/ COUNT(DISTINCT date)) AS a_clicks
  ,(AVG(a_views)/ COUNT(DISTINCT date)) AS a_views
  ,(AVG(p_cpm)/COUNT(DISTINCT date)) AS p_cpm
  ,(AVG(p_cpc)/COUNT(DISTINCT date)) AS p_cpc
  ,(AVG(p_cpv)/COUNT(DISTINCT date)) AS p_cpv
  FROM `capstone-itemized.1.t1` AS temp
  JOIN date_range 
    ON date_range.itemized_product_id = temp.itemized_product_id AND date_range.date BETWEEN DATE(temp.start_date) AND DATE(temp.end_date)
  GROUP BY itemized_product_id
)
SELECT
  date_range.date
  ,temp.media_plan_id
  ,temp.itemized_product_id
  ,temp.FlightsDimensionValueID
  ,temp.LineItemsDimensionValueID
  ,temp.PlatformsDimensionValueID
  ,temp.TacticsDimensionValueID
  ,temp.start_date
  ,temp.end_date
  ,temp.gross_rate
  ,temp.total_margin
  
  ,gross_budget_agg.gross_budget
  ,gross_budget_agg.net_budget
  ,gross_budget_agg.p_imp
  ,gross_budget_agg.p_clicks
  ,gross_budget_agg.p_views
  ,gross_budget_agg.p_cpm
  ,gross_budget_agg.p_cpc
  ,gross_budget_agg.p_cpv
  ,temp.cost_basis
FROM date_range
JOIN `capstone-pathlabs.itemized.t1` AS temp 
    ON date_range.itemized_product_id = temp.itemized_product_id AND date_range.date BETWEEN DATE(temp.start_date) AND DATE(temp.end_date)
JOIN gross_budget_agg ON temp.itemized_product_id = gross_budget_agg.itemized_product_id
